## Setup

In [35]:
import duckdb
import pandas as pd
from sqlalchemy import create_engine, text


In [36]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [37]:
conn = duckdb.connect(database='local.duckdb')

## Explore Medical Claims Table in Input Layer Schema

In [47]:
result = pd.read_sql("""select table_schema, table_name, column_name from information_schema.columns where table_schema like '%input_layer%' and table_name like '%medical_claim%'""", conn)
#result = pd.read_sql("""select table_schema, table_name, column_name from information_schema.columns where table_schema like table_name like '%place_of_service%'""", conn)


display(result.head(20))

/var/folders/95/l2wbpld92rjc9xqmcbdfsml80000gn/T/ipykernel_86823/2169171986.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql("""select table_schema, table_name, column_name from information_schema.columns where table_schema like '%input_layer%' and table_name like '%medical_claim%'""", conn)


,table_schema,table_name,column_name
0,input_layer,input_layer__medical_claim,claim_id
1,input_layer,input_layer__medical_claim,claim_line_number
2,input_layer,input_layer__medical_claim,claim_type
3,input_layer,input_layer__medical_claim,person_id
4,input_layer,input_layer__medical_claim,member_id
5,input_layer,input_layer__medical_claim,payer
6,input_layer,input_layer__medical_claim,plan
7,input_layer,input_layer__medical_claim,claim_start_date
8,input_layer,input_layer__medical_claim,claim_end_date
9,input_layer,input_layer__medical_claim,claim_line_start_date


## Join Diagnosis Terminology with Medical Claims and build Staging Dataframe

In [91]:
cancer_billing_raw = pd.read_sql("""
                     
with cancer_icd10 as (
select  icd_10_cm, long_description from terminology.icd_10_cm where icd_10_cm ~ '^C[0-9]{2}'
)
                     
select * from (select
'dx1' as dx_num,    
person_id,
claim_id,
claim_end_date,
admission_date,
discharge_date,
adt.admit_type_description,
pos.place_of_service_description,
in_network_flag,          
paid_amount,
allowed_amount,
charge_amount,
coinsurance_amount,
copayment_amount,
deductible_amount,
total_cost_amount,
diagnosis_code_1 as dx_code,
icd1.long_description as Cancer_Type,                                 
from input_layer.medical_claim mc
inner join cancer_icd10 icd1 on mc.diagnosis_code_1 = icd1.icd_10_cm
left join terminology.place_of_service pos on mc.place_of_service_code = pos.place_of_service_code
left join terminology.admit_type adt on mc.admit_type_code = adt.admit_type_code
UNION
select  
'dx2' as dx_num,   
person_id,
claim_id, 
claim_end_date,
admission_date,
discharge_date,
adt.admit_type_description,
pos.place_of_service_description,
in_network_flag,          
paid_amount,
allowed_amount,
charge_amount,
coinsurance_amount,
copayment_amount,
deductible_amount,
total_cost_amount,
diagnosis_code_2 as dx_code,
icd2.long_description as Cancer_Type,                                 
from input_layer.medical_claim mc
inner join cancer_icd10 icd2 on mc.diagnosis_code_2 = icd2.icd_10_cm
left join terminology.place_of_service pos on mc.place_of_service_code = pos.place_of_service_code
left join terminology.admit_type adt on mc.admit_type_code = adt.admit_type_code
UNION
select 
'dx3' as dx_num,   
person_id,
claim_id,
claim_end_date,
admission_date,
discharge_date,
adt.admit_type_description,
pos.place_of_service_description,  
in_network_flag,           
paid_amount,
allowed_amount,
charge_amount,
coinsurance_amount,
copayment_amount,
deductible_amount,
total_cost_amount,
diagnosis_code_3 as dx_code,
icd3.long_description as Cancer_Type,                                 
from input_layer.medical_claim mc
inner join cancer_icd10 icd3 on mc.diagnosis_code_3 = icd3.icd_10_cm
left join terminology.place_of_service pos on mc.place_of_service_code = pos.place_of_service_code
left join terminology.admit_type adt on mc.admit_type_code = adt.admit_type_code
UNION
select
'dx4' as dx_num,       
person_id,
claim_id,
claim_end_date,
admission_date,
discharge_date,
adt.admit_type_description,
pos.place_of_service_description,  
in_network_flag,              
paid_amount,
allowed_amount,
charge_amount,
coinsurance_amount,
copayment_amount,
deductible_amount,
total_cost_amount,
diagnosis_code_4 as dx_code,
icd4.long_description as Cancer_Type,                                 
from input_layer.medical_claim mc
inner join cancer_icd10 icd4 on mc.diagnosis_code_4 = icd4.icd_10_cm
left join terminology.place_of_service pos on mc.place_of_service_code = pos.place_of_service_code
left join terminology.admit_type adt on mc.admit_type_code = adt.admit_type_code
UNION 
select  
'dx5' as dx_num,    
person_id,
claim_id, 
claim_end_date,
admission_date,
discharge_date,
adt.admit_type_description,
pos.place_of_service_description,  
in_network_flag,              
paid_amount,
allowed_amount,
charge_amount,
coinsurance_amount,
copayment_amount,
deductible_amount,
total_cost_amount,
diagnosis_code_5 as dx_code,
icd5.long_description as Cancer_Type,                                 
from input_layer.medical_claim mc
inner join cancer_icd10 icd5 on mc.diagnosis_code_5 = icd5.icd_10_cm
left join terminology.place_of_service pos on mc.place_of_service_code = pos.place_of_service_code
left join terminology.admit_type adt on mc.admit_type_code = adt.admit_type_code
UNION
select 
'dx6' as dx_num,   
person_id,
claim_id,
claim_end_date,
admission_date,
discharge_date,
adt.admit_type_description,
pos.place_of_service_description,  
in_network_flag,              
paid_amount,
allowed_amount,
charge_amount,
coinsurance_amount,
copayment_amount,
deductible_amount,
total_cost_amount,
diagnosis_code_6 as dx_code,
icd6.long_description as Cancer_Type,                                 
from input_layer.medical_claim mc
inner join cancer_icd10 icd6 on mc.diagnosis_code_6 = icd6.icd_10_cm
left join terminology.place_of_service pos on mc.place_of_service_code = pos.place_of_service_code
left join terminology.admit_type adt on mc.admit_type_code = adt.admit_type_code
)
order by person_id
  """, conn)

display(cancer_billing_raw.head(20))

/var/folders/95/l2wbpld92rjc9xqmcbdfsml80000gn/T/ipykernel_86823/3786011922.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cancer_billing_raw = pd.read_sql("""


,dx_num,person_id,claim_id,claim_end_date,admission_date,discharge_date,admit_type_description,place_of_service_description,in_network_flag,paid_amount,allowed_amount,charge_amount,coinsurance_amount,copayment_amount,deductible_amount,total_cost_amount,dx_code,Cancer_Type
0,dx3,10000,890942,2018-11-11,None,None,None,Office,1,16.110001,19.400000,19.400000,None,None,None,None,C61,Malignant neoplasm of prostate
1,dx3,10000,890942,2018-11-11,None,None,None,Office,1,34.340000,44.830002,44.830002,None,None,None,None,C61,Malignant neoplasm of prostate
2,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,7.740000,8.220000,8.220000,None,None,None,None,C61,Malignant neoplasm of prostate
3,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,18.440001,18.240000,18.240000,None,None,None,None,C61,Malignant neoplasm of prostate
4,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,4.580000,3.920000,3.920000,None,None,None,None,C61,Malignant neoplasm of prostate
5,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,17.700001,19.030001,19.030001,None,None,None,None,C61,Malignant neoplasm of prostate
6,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,20.010000,20.750000,20.750000,None,None,None,None,C61,Malignant neoplasm of prostate
7,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,12.780000,13.320000,13.320000,None,None,None,None,C61,Malignant neoplasm of prostate
8,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,4.500000,4.730000,4.730000,None,None,None,None,C61,Malignant neoplasm of prostate
9,dx3,10025,1392009,2018-05-02,None,None,None,Independent Laboratory,1,0.000000,0.000000,0.000000,None,None,None,None,C61,Malignant neoplasm of prostate


## Total Claims per Cancer Type

In [99]:
dx_claims = cancer_billing_raw[['claim_id','Cancer_Type']].groupby('Cancer_Type').agg(
    Total_Claims=('claim_id', 'count'))
new_claims = dx_claims.merge(cancer_groups,on='Cancer_Type')
dx_claims

,Total_Claims
Cancer_Type,
Malignant neoplasm of base of tongue,2
Malignant neoplasm of prostate,1068
Malignant neoplasm of rectum,12
Malignant neoplasm of thymus,1
Malignant neoplasm of thyroid gland,275
"Malignant neoplasm of uterus, part unspecified",2


## Costs Per Cancer Type Per Place of Service

In [96]:
cancer_groups = cancer_billing_raw[['claim_id','Cancer_Type','place_of_service_description','paid_amount','allowed_amount', 'charge_amount']].groupby(['Cancer_Type','place_of_service_description' ]).agg(
    Total_Paid_Amount=('paid_amount', 'sum'),
    Total_Allowed_Amount=('allowed_amount', 'sum'),
    Total_Charge_Amount=('charge_amount','sum'),
    Claim_Count=('Cancer_Type', 'count')).sort_values(['Cancer_Type','place_of_service_description'])
cancer_groups

Total_Paid_Amount  \
Cancer_Type                                    place_of_service_description                        
Malignant neoplasm of base of tongue           Office                                 214.760006   
Malignant neoplasm of prostate                 Ambulance - Land                        72.939999   
                                               Ambulatory Surgical Center             999.389999   
                                               Emergency Room – Hospital              143.380005   
                                               Independent Laboratory                5770.390011   
                                               Inpatient Hospital                      87.299999   
                                               Off Campus-Outpatient Hospital         492.730003   
                                               Office                               55690.800142   
                                               On Campus-Outpatient Hospital        10231.510024   
                                               Skilled Nursing Facility                68.720001   
Malignant neoplasm of rectum                   Independent Laboratory                 142.719997   
                                               Office                                 260.860001   
                                               On Campus-Outpatient Hospital         1483.150024   
Malignant neoplasm of thymus                   Inpatient Hospital                       8.420000   
Malignant neoplasm of thyroid gland            Independent Laboratory                2441.670004   
                                               Inpatient Hospital                    5398.700017   
                                               Off Campus-Outpatient Hospital         497.760006   
                                               Office                                2984.769999   
                                               On Campus-Outpatient Hospital         1220.319998   
Malignant neoplasm of uterus, part unspecified Office                                 120.730003   

                                                                               Total_Allowed_Amount  \
Cancer_Type                                    place_of_service_description                           
Malignant neoplasm of base of tongue           Office                                    278.270004   
Malignant neoplasm of prostate                 Ambulance - Land                           90.520000   
                                               Ambulatory Surgical Center               1464.710022   
                                               Emergency Room – Hospital                 137.740005   
                                               Independent Laboratory                   5465.880003   
                                               Inpatient Hospital                        109.010001   
                                               Off Campus-Outpatient Hospital            846.820008   
                                               Office                                  47109.950010   
                                               On Campus-Outpatient Hospital           11546.070043   
                                               Skilled Nursing Facility                   89.839996   
Malignant neoplasm of rectum                   Independent Laboratory                    143.480000   
                                               Office                                    317.440006   
                                               On Campus-Outpatient Hospital             643.340027   
Malignant neoplasm of thymus                   Inpatient Hospital                         10.590000   
Malignant neoplasm of thyroid gland            Independent Laboratory                   2507.779998   
                                               Inpatient Hospital                       8115.430024   
                        